<a href="https://colab.research.google.com/github/AlexH20/Supervised-ML-sentiment-measures/blob/main/Merge_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_market_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 4.8 MB/s 
     |████████████████████████████████| 152 kB 19.1 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.3-py3-none-any.whl size=182636 sha256=04fc6b22c9d5010d08011bafd8340989eb74016d509798f82db2138b14653ee4
  Stored in directory: /root/.cache/pip/wheels/b8/a3/19/b4611514d34ffd61d13aef10fefc2dcaf3754145121ceba647
Successfully built exchange-calendars


In [ ]:
import pandas as pd
import csv
from datetime import datetime, timedelta, time
from google.colab import drive 
import sys
import re
import math
import holidays
import pandas_market_calendars as mcal

drive.mount("/content/gdrive")

#Functions to concatenate text

def concatenate_text(g):
    return ' '.join(g.Text)

#Function to count words of text using regex

def count_words(Text):
    word_count = len(re.findall(r'\w+', Text))
    return word_count

# Function to check whether date is on a non-trading day or news article is published later than 4pm. If so, then return next trading day.

holidaysUS = holidays.US()
nyse = mcal.get_calendar('NYSE')
stock_holidays = nyse.holidays()

stock_holidays = list(pd.to_datetime(stock_holidays.holidays))
stock_holidays = [x.date() for x in stock_holidays]

def check_tradingdayhour(day):
    trading_day = day
    if trading_day.hour >= 16:
        trading_day += timedelta(1)
    #Check if news article published on weekend. If so return monday. If day on holiday return next trading day
    while trading_day.weekday() in holidays.WEEKEND or trading_day in stock_holidays:
        trading_day += timedelta(1)
    return trading_day

output_file_path = "gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/"
output_file_name = "data_whole_woScAR.csv"

data_text = pd.read_csv("gdrive/My Drive/Thesis/processed data/processdata_woSWandPS.csv")

#CSV file from get_paneldata notebook
df = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/Data_wcontrol/data_whole_woScAR.csv")


#Get both dfs to same date format. Later used for merging

data_text["Date"] = pd.to_datetime(data_text["Date"])
data_text["Date"] = [check_tradingdayhour(x) for x in data_text["Date"]]
data_text["Date"] = [x.strftime("%Y-%m-%d") for x in data_text["Date"]]

df["Date"] = pd.to_datetime(df["Date"])
df["Date"] = [check_tradingdayhour(x) for x in df["Date"]]
df["Date"] = [x.strftime("%Y-%m-%d") for x in df["Date"]]

#Merge rows with multiple news article issue of the same company on a given day. 
#This can be done either for the processed text or unprocessed text. Adjust variable in concatenate_text function
new_df = data_text.groupby(["Date", "Ticker"]).apply(concatenate_text).to_frame(name = "c_Text")
df = df.merge(new_df, how= "left", left_on = ["Date", "Ticker"], right_on = ["Date", "Ticker"])

df["c_Text"] = df["c_Text"].fillna(" ")

df["word_count"] = [count_words(x) for x in df.c_Text]

df.drop_duplicates(inplace = True)
df = df.rename(columns={'c_Text': 'Text'})
df_columns = ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM","pref_alpha", "Text", "word_count", "AR", "Return"] 
df = df.reindex(columns = df_columns, inplace = True)

with open(output_file_path + output_file_name, "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(
        ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM",
         "pref_alpha", "Text", "word_count", "AR", "Return"])
    for index, row in df.iterrows():
        writer.writerow(row)






Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
